In [ ]:
import numpy as np
import scipy as sc
import pandas as pd
import ReadFromDB as RDB
from Authenticator import read_auth

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# plt.rcParams['figure.figsize'] = (20,4)

import sklearn as sk
from statsmodels.tsa.arima_model import ARIMA

# Time conversions
from datetime import datetime
convert = lambda x : datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# reload after changes have been made
from importlib import reload
reload(RDB)
for i in dir(RDB)[8:]:
    print(i,end=', ')

**An Arima time series will be used to generate a plot for each station**

In [ ]:
static_data = RDB.get_static_data()
station_numbers = list(static_data['number'].astype(int))

In [ ]:
weather = RDB.read_weather(5)
# weather.head(1)

In [101]:
station_data = RDB.station(5)
station_data.head(1)

,status,bike_stands,available_bike_stands,available_bikes,last_updated
0,OPEN,40,38,2,1.550592e+12


In [103]:
# remove any data where the station is closed
station_data = station_data[station_data.status=='OPEN']

# convert timestamps from milliseconds to seconds and change type to int
make_timestamp = lambda x: int(x/1000)
station_data.last_updated = station_data.last_updated.apply(make_timestamp)

#convert the timestamps to date_times
station_data.last_updated.apply(convert)
plt.plot(pd.to_datetime(station_data.index))

# set timestamp as index
station_data.set_index('last_updated', inplace=True)

AttributeError: 'DataFrame' object has no attribute 'last_updated'

In [ ]:
f, axes = plt.subplots(1,2,sharey=True, figsize=(20,5))

axes[0].set_title('Bike Stands')
axes[0].plot(station_data.available_bike_stands, '.')


axes[1].set_title('Bikes')
axes[1].plot(station_data.available_bikes,'.')

In [ ]:
pd.plotting.autocorrelation_plot(station_data.available_bikes)

Autocorrelation plots are often used for checking randomness in time series. This is done by computing autocorrelations for data values at varying time lags. If time series is random, such autocorrelations should be near zero for any and all time-lag separations. If time series is non-random then one or more of the autocorrelations will be significantly non-zero. The horizontal lines displayed in the plot correspond to 95% and 99% confidence bands. The dashed line is 99% confidence band. See the Wikipedia entry for more about autocorrelation plots.

**Model Application**

In [ ]:
# splitting Model data at skip point.


model = ARIMA(station_data.available_bikes, order=(5,0,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.xticks(rotation = 25)

In [ ]:
residuals.plot(kind='kde')
print(residuals.describe())